## import previous file

In [64]:
import pandas as pd
from pathlib import Path

p = Path("/Users/dengshuyue/Desktop/SDOH/analysis/output/cov_addv4_99_23.parquet")
df_my_cov_aligned_short = pd.read_parquet(p)  # uses pyarrow/fastparquet if available
print(df_my_cov_aligned_short.shape)
df_my_cov_aligned_short.head()


(128809, 68)


,SEQN,SDDSRVYR,sdmvpsu,sdmvstra,RIDAGEYR,SEX,RACE,re,household_size,DMDHHSIZ,...,HOQ065_structural_missing,household_size_structural_missing,chol_rx_structural_missing,RIAGENDR,drinking,alcg2,perE_alco,METSCORE_fromPAQ,LTPA_fromPAQ,met_hr_recalc_from
0,1,1.0,1,5,2.0,F,4.0,Other Hispanic,3,3.0,...,False,False,False,2.0,NaN,<NA>,0.000000,NaN,NaN,None
1,2,1.0,3,1,77.0,M,3.0,Mexican American,1,1.0,...,False,False,False,1.0,0.065753,2,0.000000,NaN,NaN,from_new_PAQ
2,3,1.0,2,7,10.0,F,3.0,Mexican American,4,4.0,...,False,False,False,2.0,NaN,<NA>,0.000000,NaN,NaN,None
3,4,1.0,1,2,1.0,M,4.0,Other Hispanic,7,7.0,...,False,False,False,1.0,NaN,<NA>,0.000000,NaN,NaN,None
4,5,1.0,2,8,49.0,M,3.0,Mexican American,3,3.0,...,False,False,False,1.0,1.714286,2,9.101101,NaN,NaN,from_new_PAQ


## show missing 

In [65]:
df = df_my_cov_aligned_short

# use every column except the grouper
# ignore PACK_YEARS, CIGS_PER_DAY, probable_depression and etc as missing naturally 
# ignore dulicate safe saved old column name

exclude = {"SDDSRVYR","CIGS_PER_DAY","PACK_YEARS","probable_depression","wt_phlebotomy", "WTSAFPRP",
           "WTINT2YR", "WTMEC2YR", "WTPH2YR", "WTSAF2YR", "WTMEC4YR", "WTINTPRP", "WTMECPRP", "WTINT4YR",
           "SNAP", "SNAP_src", "SNAP_bin", "SNAP_src_rank", "bmi", "RIAGENDR",
           "SNAP_indiv_only","FS", "ahei_total", "HOQ065", "marriage_label", "marriage_prev",
           "METSCORE_fromPAQ","perE_alco","LTPA_fromPAQ","SNAP_indiv_plus_singleton", 
           "SMK_AVG", "BMI_CLAS", "household_size", "DMDHHSIZ"}
cols_all = [c for c in df.columns if c not in exclude]

# % missing by cycle (split into two lines)
is_na = df[cols_all].isna()
pct_miss = is_na.groupby(df["SDDSRVYR"]).mean().mul(100)

# keep only columns that exceed 80% missing in ANY cycle
pct_miss_gt80 = pct_miss.loc[:, (pct_miss > 80).any(axis=0)].round(1)

print(pct_miss_gt80)


          chol_rx  alcg2
SDDSRVYR                
1.0           0.0   62.9
2.0           0.0   66.5
3.0           0.0   62.9
4.0           0.0   62.7
5.0           0.0   55.4
6.0           0.0   55.4
7.0           0.0   55.1
8.0           0.0   52.2
9.0           0.0   52.8
10.0          0.0   56.0
12.0         29.1   61.4
66.0        100.0   84.3


## fetch chol and code chol_rx

#### check if previous cov code have mention chol_rx method

In [66]:
from pathlib import Path
import re

"""
p = Path("/Users/dengshuyue/Desktop/SDOH/analysis/2.1_Prepare data_covariates.sas")
pat = re.compile(r'chol(_rx)?|HIGH_CHOL|BPQ0?80|BPQ10(0D|1D)|LBXTC|LBDLDL|LBXTR|HDL|RXQ_RX|RXDDRUG|STATIN|EZETIMIBE|FIBRATE', re.I)

lines = p.read_text(errors="ignore").splitlines()
for i, line in enumerate(lines):
    if pat.search(line):
        start = max(0, i-3); end = min(len(lines), i+4)
        print(f"\n--- lines {start+1}-{end} ---")
        for j in range(start, end):
            print(f"{j+1:5d}: {lines[j]}")
"""


'\np = Path("/Users/dengshuyue/Desktop/SDOH/analysis/2.1_Prepare data_covariates.sas")\npat = re.compile(r\'chol(_rx)?|HIGH_CHOL|BPQ0?80|BPQ10(0D|1D)|LBXTC|LBDLDL|LBXTR|HDL|RXQ_RX|RXDDRUG|STATIN|EZETIMIBE|FIBRATE\', re.I)\n\nlines = p.read_text(errors="ignore").splitlines()\nfor i, line in enumerate(lines):\n    if pat.search(line):\n        start = max(0, i-3); end = min(len(lines), i+4)\n        print(f"\n--- lines {start+1}-{end} ---")\n        for j in range(start, end):\n            print(f"{j+1:5d}: {lines[j]}")\n'

#### Full cycle fetch 99-23

In [67]:
import io, re, requests
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype

# =========================================================
# Helpers
# =========================================================
def _read_xpt(url, cols_upper=True):
    r = requests.get(url); r.raise_for_status()
    df = pd.read_sas(io.BytesIO(r.content), format="xport", encoding="latin1")
    if cols_upper:
        df.columns = [c.upper() for c in df.columns]
    if "SEQN" in df.columns:
        df["SEQN"] = pd.to_numeric(df["SEQN"], errors="coerce").astype("Int64")
    return df

def _try_read_xpt(url, cols_upper=True):
    try:
        if not url:
            return pd.DataFrame()
        return _read_xpt(url, cols_upper=cols_upper)
    except requests.HTTPError:
        return pd.DataFrame()

def _find_numeric_col(df, preferred, fallback_regex):
    if df is None or df.empty:
        return None
    for c in preferred:
        if c in df.columns and is_numeric_dtype(df[c]):
            return c
    pats = re.compile(fallback_regex, flags=re.I)
    for c in df.columns:
        if pats.search(c) and is_numeric_dtype(df[c]):
            return c
    return None

def _friedewald_ldl(tc, hdl, tg):
    """LDL = TC - HDL - TG/5 when TG < 400; else NA."""
    ldl = pd.Series(pd.NA, index=tc.index, dtype="Float64")
    cond = tc.notna() & hdl.notna() & tg.notna() & (tg < 400)
    ldl.loc[cond] = (tc.loc[cond] - hdl.loc[cond] - (tg.loc[cond] / 5.0))
    return ldl

def wmean(x, w):
    x = pd.to_numeric(x, errors="coerce")
    w = pd.to_numeric(w, errors="coerce")
    m = x.notna() & w.notna() & (w > 0)
    if m.sum() == 0: return np.nan
    return float((x[m] * w[m]).sum() / w[m].sum())

# =========================================================
# File plumbing (1999–2018 two-year cycles)
# =========================================================
CYCLES_2YR = [
    ("",   "1999-2000"),
    ("_B", "2001-2002"),
    ("_C", "2003-2004"),
    ("_D", "2005-2006"),
    ("_E", "2007-2008"),
    ("_F", "2009-2010"),
    ("_G", "2011-2012"),
    ("_H", "2013-2014"),
    ("_I", "2015-2016"),
    ("_J", "2017-2018"),
]

def _base_folder_2yr(suffix):
    return {
        "":   "1999", "_B": "2001", "_C": "2003", "_D": "2005", "_E": "2007",
        "_F": "2009", "_G": "2011", "_H": "2013", "_I": "2015", "_J": "2017"
    }[suffix]

def _url_2yr(folder, filebase):
    return f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/{folder}/DataFiles/{filebase}.xpt"

def _read_first(folder, bases):
    for base in bases:
        df = _try_read_xpt(_url_2yr(folder, base))
        if not df.empty:
            return df
    return pd.DataFrame()

# candidates per cycle for TC, HDL, LDL/TG
def _tc_candidates(suffix):
    if suffix == "":
        return ["LAB13", "TCHOL"]                 # 1999–2000
    if suffix in {"_B","_C"}:
        s = suffix.lstrip("_")
        return [f"L13_{s}", f"TCHOL{suffix}"]     # 2001–2004
    return [f"TCHOL{suffix}"]                     # 2005+

def _hdl_candidates(suffix):
    if suffix == "":
        return ["LAB13", "HDL"]                   # 1999–2000
    if suffix in {"_B","_C"}:
        s = suffix.lstrip("_")
        return [f"L13_{s}", f"HDL{suffix}"]       # 2001–2004
    return [f"HDL{suffix}"]                       # 2005+

def _ldl_tg_candidates(suffix):
    if suffix == "":
        return ["LAB13AM", "TRIGLY", "TCHOL"]     # 1999–2000
    if suffix in {"_B","_C"}:
        s = suffix.lstrip("_")
        return [f"L13AM_{s}", f"TRIGLY{suffix}", f"TCHOL{suffix}"]  # 2001–2004
    return [f"TRIGLY{suffix}", f"TCHOL{suffix}"] # 2005+

# =========================================================
# Build ONE 2-yr cycle (SAS: dx = 080 OR 090D; flag = dx OR (TC>200))
# =========================================================
def build_cycle_2yr(suffix: str, label: str) -> pd.DataFrame:
    folder = _base_folder_2yr(suffix)

    # BPQ (dx items)
    bpq = _try_read_xpt(_url_2yr(folder, f"BPQ{suffix}"))
    bpq_keep = pd.DataFrame()
    if not bpq.empty:
        bpq_keep = bpq[["SEQN"]].copy()
        has_080 = pd.to_numeric(bpq.get("BPQ080"),  errors="coerce").eq(1) if "BPQ080"  in bpq.columns else pd.Series(False, index=bpq.index)
        has_090 = pd.to_numeric(bpq.get("BPQ090D"), errors="coerce").eq(1) if "BPQ090D" in bpq.columns else pd.Series(False, index=bpq.index)
        bpq_keep["hc_dx"] = (has_080 | has_090).astype("Int64")

    # Labs (TC and HDL from their own sources; LDL/TG from LDL/TG source)
    tc_df  = _read_first(folder, _tc_candidates(suffix))
    hdl_df = _read_first(folder, _hdl_candidates(suffix))
    ldl_tg = _read_first(folder, _ldl_tg_candidates(suffix))

    # TC
    tc_var  = _find_numeric_col(
        tc_df,
        preferred=["LBXTC"],                      # total cholesterol
        fallback_regex=r"\bTOTAL.*CHOL|^LB..TC$|\bTC\b"
    )
    # HDL — prefer DIRECT (LBDHDD) when present, else conventional (LBDHDL/LBXHDL), else older naming (LBXHDD)
    hdl_var = _find_numeric_col(
        hdl_df,
        preferred=["LBDHDD","LBDHDL","LBXHDL","LBXHDD","LBDHDD_1"],
        fallback_regex=r"\bHDL\b|HDDS?\b|^LB..HD"
    )
    # LDL — look in TC file first, then LDL/TG file
    ldl_var = _find_numeric_col(
        tc_df,
        preferred=["LBDLDL","LBDLDLD","LBDLDLL"],
        fallback_regex=r"\bLDL\b|^LB.DLDL"
    )
    if ldl_var is None:
        ldl_var = _find_numeric_col(
            ldl_tg,
            preferred=["LBDLDL","LBDLDLD","LBDLDLL"],
            fallback_regex=r"\bLDL\b|^LB.DLDL"
        )
    # TG
    tg_var  = _find_numeric_col(
        ldl_tg,
        preferred=["LBXTR"],
        fallback_regex=r"\bTRI?G|^LB..TR$"
    )

    # Keep frames
    tc_keep  = tc_df[["SEQN", tc_var ]].rename(columns={tc_var : "total_chol_mgdl"}) if tc_var  else pd.DataFrame(columns=["SEQN","total_chol_mgdl"])
    hdl_keep = hdl_df[["SEQN", hdl_var]].rename(columns={hdl_var: "hdl_mgdl"})       if hdl_var else pd.DataFrame(columns=["SEQN","hdl_mgdl"])
    ldl_keep = (tc_df if (ldl_var and ldl_var in tc_df.columns) else ldl_tg)
    ldl_keep = ldl_keep[["SEQN", ldl_var]].rename(columns={ldl_var: "ldl_mgdl"})     if ldl_var else pd.DataFrame(columns=["SEQN","ldl_mgdl"])
    tg_keep  = ldl_tg[["SEQN", tg_var]].rename(columns={tg_var: "trig"})             if tg_var  else pd.DataFrame(columns=["SEQN","trig"])

    # Merge universe (union of sources)
    bases = [f for f in (bpq_keep, tc_keep, hdl_keep, ldl_keep, tg_keep) if not f.empty]
    if not bases:
        return pd.DataFrame(columns=["SEQN","period","chol_rx2","hc_dx","hc_lab200","total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"])
    base = bases[0][["SEQN"]].copy()
    for f in bases[1:]:
        base = base.merge(f[["SEQN"]], on="SEQN", how="outer")

    df = base
    if not bpq_keep.empty: df = df.merge(bpq_keep, how="left", on="SEQN")
    if not tc_keep.empty:  df = df.merge(tc_keep,  how="left", on="SEQN")
    if not hdl_keep.empty: df = df.merge(hdl_keep, how="left", on="SEQN")
    if not ldl_keep.empty: df = df.merge(ldl_keep, how="left", on="SEQN")
    if not tg_keep.empty:  df = df.merge(tg_keep,  how="left", on="SEQN")

    # Normalize labs
    for c in ["total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]:
        if c not in df.columns: df[c] = pd.NA
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("Float64")

    # Friedewald LDL (summaries only)
    need_calc = df["ldl_mgdl"].isna()
    if need_calc.any() and df["total_chol_mgdl"].notna().any() and df["hdl_mgdl"].notna().any() and df["trig"].notna().any():
        ldl_est = _friedewald_ldl(df["total_chol_mgdl"], df["hdl_mgdl"], df["trig"])
        df.loc[need_calc, "ldl_mgdl"] = ldl_est.loc[need_calc]

    # SAS rule
    df["hc_lab200"] = (df["total_chol_mgdl"] > 200).astype("Int64")
    dx  = pd.to_numeric(df.get("hc_dx"),     errors="coerce").fillna(0).astype(int)
    lab = pd.to_numeric(df.get("hc_lab200"), errors="coerce").fillna(0).astype(int)
    df["chol_rx2"] = ((dx == 1) | (lab == 1)).astype(int)

    df["period"] = label
    return df[["SEQN","period","chol_rx2","hc_dx","hc_lab200","total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]]

# =========================================================
# P & L combined cycles (2017–2020, 2021–2022)
# =========================================================
BASE_2017 = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles"  # 'P' combined
BASE_2021 = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles"  # 'L'

def _url_P(fn): return f"{BASE_2017}/{fn}.xpt"
def _url_L(fn): return f"{BASE_2021}/{fn}.xpt"

def build_P() -> pd.DataFrame:
    # BPQ_P with dx = BPQ080 OR BPQ090D (SAS)
    bpq = _read_xpt(_url_P("P_BPQ"))
    bpq_keep = bpq[["SEQN"]].copy()
    has_080 = pd.to_numeric(bpq.get("BPQ080"),  errors="coerce").eq(1) if "BPQ080" in bpq.columns else pd.Series(False, index=bpq.index)
    has_090 = pd.to_numeric(bpq.get("BPQ090D"), errors="coerce").eq(1) if "BPQ090D" in bpq.columns else pd.Series(False, index=bpq.index)
    bpq_keep["hc_dx"] = (has_080 | has_090).astype("Int64")

    # Labs: separate files
    tch  = _read_xpt(_url_P("P_TCHOL"))
    hdl  = _read_xpt(_url_P("P_HDL"))
    trig = _try_read_xpt(_url_P("P_TRIGLY"))

    tc_var  = _find_numeric_col(tch, preferred=["LBXTC"], fallback_regex=r"\bTOTAL.*CHOL|^LB..TC$|\bTC\b")
    hdl_var = _find_numeric_col(hdl, preferred=["LBDHDD","LBDHDL","LBXHDL","LBXHDD","LBDHDD_1"], fallback_regex=r"\bHDL\b|HDDS?\b|^LB..HD")
    ldl_var = _find_numeric_col(tch, preferred=["LBDLDL","LBDLDLD","LBDLDLL"], fallback_regex=r"\bLDL\b|^LB.DLDL")
    tg_var  = _find_numeric_col(trig, preferred=["LBXTR"], fallback_regex=r"\bTRI?G|^LB..TR$") if not trig.empty else None

    tc_keep  = tch[["SEQN", tc_var ]].rename(columns={tc_var : "total_chol_mgdl"}) if tc_var  else pd.DataFrame(columns=["SEQN","total_chol_mgdl"])
    hdl_keep = hdl[["SEQN", hdl_var]].rename(columns={hdl_var: "hdl_mgdl"})         if hdl_var else pd.DataFrame(columns=["SEQN","hdl_mgdl"])
    ldl_keep = tch[["SEQN", ldl_var]].rename(columns={ldl_var: "ldl_mgdl"})         if ldl_var else pd.DataFrame(columns=["SEQN","ldl_mgdl"])
    tg_keep  = trig[["SEQN", tg_var]].rename(columns={tg_var: "trig"})              if tg_var  else pd.DataFrame(columns=["SEQN","trig"])

    bases = [f for f in (bpq_keep, tc_keep, hdl_keep, ldl_keep, tg_keep) if not f.empty]
    base = bases[0][["SEQN"]].copy()
    for f in bases[1:]:
        base = base.merge(f[["SEQN"]], on="SEQN", how="outer")

    df = base.merge(bpq_keep, how="left", on="SEQN")
    for f in (tc_keep, hdl_keep, ldl_keep, tg_keep):
        df = df.merge(f, how="left", on="SEQN")

    for c in ["total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]:
        if c not in df.columns: df[c] = pd.NA
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("Float64")

    need_calc = df["ldl_mgdl"].isna()
    if need_calc.any() and df["total_chol_mgdl"].notna().any() and df["hdl_mgdl"].notna().any() and df["trig"].notna().any():
        df.loc[need_calc, "ldl_mgdl"] = _friedewald_ldl(df["total_chol_mgdl"], df["hdl_mgdl"], df["trig"])

    df["hc_lab200"] = (df["total_chol_mgdl"] > 200).astype("Int64")
    df["chol_rx2"]  = ((df["hc_dx"].fillna(0).astype(int)==1) | (df["hc_lab200"].fillna(0).astype(int)==1)).astype(int)
    df["period"]    = "2017-2020 (P)"
    return df[["SEQN","period","chol_rx2","hc_dx","hc_lab200","total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]]

def build_L() -> pd.DataFrame:
    # BPQ_L with dx = BPQ080 OR BPQ090D (if present)
    bpq = _read_xpt(_url_L("BPQ_L"))
    bpq_keep = bpq[["SEQN"]].copy()
    has_080 = pd.to_numeric(bpq.get("BPQ080"),  errors="coerce").eq(1) if "BPQ080" in bpq.columns else pd.Series(False, index=bpq.index)
    has_090 = pd.to_numeric(bpq.get("BPQ090D"), errors="coerce").eq(1) if "BPQ090D" in bpq.columns else pd.Series(False, index=bpq.index)
    bpq_keep["hc_dx"] = (has_080 | has_090).astype("Int64")

    # Labs
    tch  = _read_xpt(_url_L("TCHOL_L"))
    hdl  = _read_xpt(_url_L("HDL_L"))
    trig = _try_read_xpt(_url_L("TRIGLY_L"))  # may not exist

    tc_var  = _find_numeric_col(tch, preferred=["LBXTC"], fallback_regex=r"\bTOTAL.*CHOL|^LB..TC$|\bTC\b")
    hdl_var = _find_numeric_col(hdl, preferred=["LBDHDD","LBDHDL","LBXHDL","LBXHDD","LBDHDD_1"], fallback_regex=r"\bHDL\b|HDDS?\b|^LB..HD")
    ldl_var = _find_numeric_col(tch, preferred=["LBDLDL","LBDLDLD","LBDLDLL"], fallback_regex=r"\bLDL\b|^LB.DLDL")
    tg_var  = _find_numeric_col(trig, preferred=["LBXTR"], fallback_regex=r"\bTRI?G|^LB..TR$") if not trig.empty else None

    tc_keep  = tch[["SEQN", tc_var ]].rename(columns={tc_var : "total_chol_mgdl"}) if tc_var  else pd.DataFrame(columns=["SEQN","total_chol_mgdl"])
    hdl_keep = hdl[["SEQN", hdl_var]].rename(columns={hdl_var: "hdl_mgdl"})         if hdl_var else pd.DataFrame(columns=["SEQN","hdl_mgdl"])
    ldl_keep = tch[["SEQN", ldl_var]].rename(columns={ldl_var: "ldl_mgdl"})         if ldl_var else pd.DataFrame(columns=["SEQN","ldl_mgdl"])
    tg_keep  = trig[["SEQN", tg_var]].rename(columns={tg_var: "trig"})              if tg_var  else pd.DataFrame(columns=["SEQN","trig"])

    bases = [f for f in (bpq_keep, tc_keep, hdl_keep, ldl_keep, tg_keep) if not f.empty]
    base = bases[0][["SEQN"]].copy()
    for f in bases[1:]:
        base = base.merge(f[["SEQN"]], on="SEQN", how="outer")

    df = base.merge(bpq_keep, how="left", on="SEQN")
    for f in (tc_keep, hdl_keep, ldl_keep, tg_keep):
        df = df.merge(f, how="left", on="SEQN")

    for c in ["total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]:
        if c not in df.columns: df[c] = pd.NA
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("Float64")

    need_calc = df["ldl_mgdl"].isna()
    if need_calc.any() and df["total_chol_mgdl"].notna().any() and df["hdl_mgdl"].notna().any() and df["trig"].notna().any():
        df.loc[need_calc, "ldl_mgdl"] = _friedewald_ldl(df["total_chol_mgdl"], df["hdl_mgdl"], df["trig"])

    df["hc_lab200"] = (df["total_chol_mgdl"] > 200).astype("Int64")
    df["chol_rx2"]  = ((df["hc_dx"].fillna(0).astype(int)==1) | (df["hc_lab200"].fillna(0).astype(int)==1)).astype(int)
    df["period"]    = "2021-2022 (L)"
    return df[["SEQN","period","chol_rx2","hc_dx","hc_lab200","total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]]

# =========================================================
# Build 1999–2018 (2-yr), 2017–2020 (P), 2021–2022 (L)
# =========================================================
dfs_2yr = []
for suf, lab in CYCLES_2YR:
    d = build_cycle_2yr(suf, lab)
    if not d.empty:
        dfs_2yr.append(d)
df_99_18 = pd.concat(dfs_2yr, ignore_index=True) if dfs_2yr else pd.DataFrame(
    columns=["SEQN","period","chol_rx2","hc_dx","hc_lab200","total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]
)

df_P = build_P()
df_L = build_L()

df_all = pd.concat([df_99_18, df_P, df_L], ignore_index=True)

# =========================================================
# Unweighted QC
# =========================================================
qc_unw = (
    df_all.groupby("period", dropna=False)
          .agg(n=("SEQN","count"),
               chol_rx2_rate=("chol_rx2","mean"),
               dx_rate=("hc_dx","mean"),
               lab200_rate=("hc_lab200","mean"),
               mean_tc=("total_chol_mgdl","mean"),
               mean_hdl=("hdl_mgdl","mean"),
               mean_ldl=("ldl_mgdl","mean"))
          .round(3)
          .sort_index()
)
print("\n=== Unweighted QC (SAS-aligned, 1999–2022) ===")
print(qc_unw)

# =========================================================
# Weighted QC
# - 2-yr cycles: WTMEC2YR
# - P combined:  WTMECPRP (fallback WTMEC2YR/WTINTPRP/WTINT2YR)
# - L:           WTMEC2YR
# =========================================================
def _pick_weight_col(cols, cand):
    for c in cand:
        if c in cols: return c
    return None

def fetch_weight_for_period(period_label):
    if period_label in [lab for _, lab in CYCLES_2YR]:
        suffix = [s for s, lab in CYCLES_2YR if lab == period_label][0]
        folder = _base_folder_2yr(suffix)
        d = _try_read_xpt(_url_2yr(folder, f"DEMO{suffix}"))
        if d.empty: return pd.DataFrame({"SEQN": pd.Series(dtype="Int64"), "W": pd.Series(dtype="float")})
        wcol = _pick_weight_col(d.columns, ["WTMEC2YR"])
        out = d[["SEQN"]].copy()
        out["W"] = pd.to_numeric(d[wcol], errors="coerce") if wcol else 1.0
        out["_WCOL"] = wcol if wcol else "unit"
        return out

    if period_label == "2017-2020 (P)":
        d = _read_xpt(f"{BASE_2017}/P_DEMO.xpt")
        wcol = _pick_weight_col(d.columns, ["WTMECPRP","WTMEC2YR","WTINTPRP","WTINT2YR"])
        out = d[["SEQN"]].copy()
        out["W"] = pd.to_numeric(d[wcol], errors="coerce") if wcol else 1.0
        out["_WCOL"] = wcol if wcol else "unit"
        return out

    if period_label == "2021-2022 (L)":
        d = _read_xpt(f"{BASE_2021}/DEMO_L.xpt")
        wcol = _pick_weight_col(d.columns, ["WTMEC2YR","WTINT2YR"])
        out = d[["SEQN"]].copy()
        out["W"] = pd.to_numeric(d[wcol], errors="coerce") if wcol else 1.0
        out["_WCOL"] = wcol if wcol else "unit"
        return out

    return pd.DataFrame({"SEQN": pd.Series(dtype="Int64"), "W": pd.Series(dtype="float")})

rows = []
for per in sorted(df_all["period"].dropna().unique()):
    demo = fetch_weight_for_period(per)
    d = df_all[df_all["period"]==per].merge(demo, on="SEQN", how="left")
    rows.append({
        "period": per,
        "weight_used": d.get("_WCOL", pd.Series(["unknown"])).mode().iat[0] if "_WCOL" in d.columns and not d["_WCOL"].dropna().empty else "unknown",
        "w_union":   round(wmean(d["chol_rx2"],        d["W"]), 3),
        "w_dx":      round(wmean(d["hc_dx"],           d["W"]), 3),
        "w_lab200":  round(wmean(d["hc_lab200"],       d["W"]), 3),
        "w_mean_tc": round(wmean(d["total_chol_mgdl"], d["W"]), 1),
        "w_mean_hdl":round(wmean(d["hdl_mgdl"],        d["W"]), 1),
        "w_mean_ldl":round(wmean(d["ldl_mgdl"],        d["W"]), 1),
    })

qc_w = pd.DataFrame(rows).set_index("period").sort_index()
print("\n=== Weighted QC (SAS-aligned) ===")
print(qc_w[["weight_used","w_union","w_dx","w_lab200","w_mean_tc","w_mean_hdl","w_mean_ldl"]])



=== Unweighted QC (SAS-aligned, 1999–2022) ===
                   n  chol_rx2_rate  dx_rate  lab200_rate  mean_tc  mean_hdl  \
period                                                                         
1999-2000       8839          0.333    0.193        0.336  186.956    50.987   
2001-2002       9682          0.336    0.201        0.324  186.684    51.649   
2003-2004       8884          0.356    0.243        0.318   185.29    54.472   
2005-2006       8334          0.358    0.226        0.309  184.471    54.782   
2007-2008       8371          0.417    0.291        0.333  186.675    52.037   
2009-2010       8763          0.408    0.272        0.326   185.64    52.648   
2011-2012       8077          0.413    0.323         0.31  183.202    52.626   
2013-2014       8489          0.400    0.341        0.272  179.534    53.105   
2015-2016       8285          0.401    0.341        0.279  180.257    54.401   
2017-2018       7768          0.417    0.358        0.273  179.895    53

## save and label new chol_rx 

In [68]:
# Alias chol_rx2 -> chol_rx200 and merge into your covariates DF
# (keeps only one column named 'chol_rx200')

# 1) Build the add-on frame from df_all
add = (
    df_all.loc[:, ["SEQN", "chol_rx2"]]
         .dropna(subset=["SEQN"])
         .drop_duplicates(subset=["SEQN"])
         .rename(columns={"chol_rx2": "chol_rx200"})
)

# 2) Harmonize key dtype
df_my_cov_aligned_short["SEQN"] = pd.to_numeric(df_my_cov_aligned_short["SEQN"], errors="coerce").astype("Int64")
add["SEQN"] = pd.to_numeric(add["SEQN"], errors="coerce").astype("Int64")

# 3) Merge (and replace any existing chol_rx200)
df_my_cov_aligned_short = df_my_cov_aligned_short.drop(columns=["chol_rx200"], errors="ignore")
df_my_cov_aligned_short = df_my_cov_aligned_short.merge(add, on="SEQN", how="left")

# 4) Ensure nullable-int dtype
df_my_cov_aligned_short["chol_rx200"] = pd.to_numeric(df_my_cov_aligned_short["chol_rx200"], errors="coerce").astype("Int64")

# (optional) quick sanity
print(
    df_my_cov_aligned_short["chol_rx200"].agg(["count","mean"]).round(3)
)

# (optional) save
# df_my_cov_aligned_short.to_parquet("cov_addv4_99_23.parquet", index=False)


count    108394.000
mean          0.389
Name: chol_rx200, dtype: float64


## check current missingness again

In [69]:
df_my_cov_aligned_short.columns

Index(['SEQN', 'SDDSRVYR', 'sdmvpsu', 'sdmvstra', 'RIDAGEYR', 'SEX', 'RACE',
       're', 'household_size', 'DMDHHSIZ', 'EDU', 'pir', 'SMK_AVG', 'SMK',
       'met_hr', 'SMK_STATUS', 'CIGS_PER_DAY', 'PACK_YEARS', 'FORMER_SMOKER',
       'METSCORE', 'LTPA', 'bmi_cat', 'BMI_CLAS', 'DIABE', 'HYPERTEN',
       'chol_rx', 'CVD', 'cancer', 'probable_depression', 'ahei_total',
       'unemployment2', 'ins', 'HOQ065', 'marriage', 'SNAP', 'FS', 'WTINT2YR',
       'WTMEC2YR', 'WTSAF2YR', 'WTINT4YR', 'WTMEC4YR', 'WTPH2YR', 'WTINTPRP',
       'WTMECPRP', 'WTSAFPRP', 'wt_int', 'wt_mec', 'wt_fasting',
       'wt_phlebotomy', 'marriage_prev', 'marriage_label', 'marriage3',
       'SNAP_src', 'SNAP_bin', 'SNAP_src_rank', 'SNAP_indiv_only',
       'SNAP_indiv_plus_singleton', 'bmi', 'HOQ065_structural_missing',
       'household_size_structural_missing', 'chol_rx_structural_missing',
       'RIAGENDR', 'drinking', 'alcg2', 'perE_alco', 'METSCORE_fromPAQ',
       'LTPA_fromPAQ', 'met_hr_recalc_from', 'c

In [70]:
df = df_my_cov_aligned_short

# use every column except the grouper
# ignore PACK_YEARS, CIGS_PER_DAY, probable_depression and etc as missing naturally 
# ignore dulicate safe saved old column name

exclude = {"SDDSRVYR","CIGS_PER_DAY","PACK_YEARS","probable_depression","wt_phlebotomy", "WTSAFPRP",
           "WTINT2YR", "WTMEC2YR", "WTPH2YR", "WTSAF2YR", "WTMEC4YR", "WTINTPRP", "WTMECPRP", "WTINT4YR",
           "SNAP", "SNAP_src", "SNAP_bin", "SNAP_src_rank", "bmi", "RIAGENDR",
           "SNAP_indiv_only","FS", "ahei_total", "HOQ065", "marriage_label", "marriage_prev",
           "METSCORE_fromPAQ","perE_alco","LTPA_fromPAQ","SNAP_indiv_plus_singleton", 
           "SMK_AVG", "BMI_CLAS", "household_size", "DMDHHSIZ", "chol_rx", "chol_rx_structural_missing"}
cols_all = [c for c in df.columns if c not in exclude]

# % missing by cycle (split into two lines)
is_na = df[cols_all].isna()
pct_miss = is_na.groupby(df["SDDSRVYR"]).mean().mul(100)

# keep only columns that exceed 80% missing in ANY cycle
pct_miss_gt80 = pct_miss.loc[:, (pct_miss > 80).any(axis=0)].round(1)

print(pct_miss_gt80)


          alcg2
SDDSRVYR       
1.0        62.9
2.0        66.5
3.0        62.9
4.0        62.7
5.0        55.4
6.0        55.4
7.0        55.1
8.0        52.2
9.0        52.8
10.0       56.0
12.0       61.4
66.0       84.3


In [71]:
df_my_cov_aligned_short[["alcg2", "drinking"]]

,alcg2,drinking
0,<NA>,NaN
1,2,0.065753
2,<NA>,NaN
3,<NA>,NaN
4,2,1.714286
...,...,...
128804,<NA>,NaN
128805,1,0.004110
128806,2,0.049315
128807,1,0.000000


## fix final missing alg2 

In [72]:
import io, requests, numpy as np, pandas as pd

# ---------------------------
# Helper to read XPT
# ---------------------------
def _read_xpt(url, cols_upper=True):
    r = requests.get(url); r.raise_for_status()
    df = pd.read_sas(io.BytesIO(r.content), format="xport", encoding="latin1")
    if cols_upper: df.columns = [c.upper() for c in df.columns]
    return df

# ---------------------------
# (Optional) attach gender/age for SDDSRVYR==66 only (no ALQ fields)
# ---------------------------
def attach_p_demo_gender(df: pd.DataFrame, replace_existing: bool=False) -> pd.DataFrame:
    out = df.copy()
    for c in ["SEQN","SDDSRVYR","RIAGENDR","RIDAGEYR"]:
        if c not in out.columns: out[c] = np.nan

    m66 = pd.to_numeric(out["SDDSRVYR"], errors="coerce").eq(66)
    idx66 = out.index[m66]
    if idx66.empty: return out

    demo = _read_xpt("https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/P_DEMO.xpt")
    demo = demo[["SEQN","RIAGENDR","RIDAGEYR"]].copy()
    for c in ["RIAGENDR","RIDAGEYR"]:
        demo[c] = pd.to_numeric(demo[c], errors="coerce")

    part = out.loc[idx66, ["SEQN"]].merge(demo, on="SEQN", how="left")
    part.index = idx66  # align to original index

    if replace_existing:
        out.loc[idx66, ["RIAGENDR","RIDAGEYR"]] = part[["RIAGENDR","RIDAGEYR"]].values
    else:
        need_g = idx66[out.loc[idx66, "RIAGENDR"].isna()]
        need_a = idx66[out.loc[idx66, "RIDAGEYR"].isna()]
        if len(need_g): out.loc[need_g, "RIAGENDR"] = part.loc[need_g, "RIAGENDR"].values
        if len(need_a): out.loc[need_a, "RIDAGEYR"] = part.loc[need_a, "RIDAGEYR"].values
    return out

# ---------------------------
# Compute alcg2 for cycle 66 WITHOUT persisting ALQ columns
# Categories:
#   1 = none/rare  (drinks/day < 0.03)
#   2 = light/mod  (male: 0.03–<2; female: 0.03–<1)
#   3 = heavy      (male: ≥2; female: ≥1)
# ---------------------------
def update_alcg2_for_p_cycle_no_intermediates(
    df: pd.DataFrame,
    replace_existing: bool=False,
    rare_cutoff_per_day: float=0.03,
    backfill_alq130_if_freq: bool=True
) -> pd.DataFrame:

    out = df.copy()
    for c in ["SEQN","SDDSRVYR","RIDAGEYR","RIAGENDR","alcg2"]:
        if c not in out.columns: out[c] = np.nan
    if out["alcg2"].dtype.name != "Int64":
        out["alcg2"] = pd.Series(out["alcg2"]).astype("Int64")

    sdd   = pd.to_numeric(out["SDDSRVYR"], errors="coerce")
    age18 = pd.to_numeric(out["RIDAGEYR"], errors="coerce").ge(18)
    idx   = out.index[sdd.eq(66) & age18]
    if idx.empty: return out

    # Read ALQ for this cycle, but DO NOT add to df
    alq = _read_xpt("https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/P_ALQ.xpt")
    keep = [c for c in ["SEQN","ALQ111","ALQ121","ALQ130"] if c in alq.columns]
    alq = alq[keep].copy()
    for c in ["ALQ111","ALQ121","ALQ130"]:
        if c in alq.columns: alq[c] = pd.to_numeric(alq[c], errors="coerce")
    if "ALQ130" in alq.columns:
        alq["ALQ130"] = alq["ALQ130"].replace({777: np.nan, 999: np.nan})

    work = out.loc[idx, ["SEQN","RIAGENDR","alcg2"]].merge(alq, on="SEQN", how="left")
    work.index = idx

    if backfill_alq130_if_freq and {"ALQ111","ALQ121","ALQ130"} <= set(work.columns):
        cond = (work["ALQ111"] == 1) & (work["ALQ121"] > 0) & (work["ALQ130"].isna())
        work.loc[cond, "ALQ130"] = 1.0

    freq_map = {0:0.0, 1:1.0, 2:6/7, 3:3.5/7, 4:2/7, 5:1/7, 6:1/12.5, 7:1/30, 8:9/365, 9:4.5/365, 10:1.5/365}
    mult = work["ALQ121"].map(freq_map) if "ALQ121" in work.columns else pd.Series(np.nan, index=work.index)

    drinks_per_day = mult * work.get("ALQ130", np.nan)
    if "ALQ111" in work.columns:
        drinks_per_day = np.where(work["ALQ111"] == 2, 0.0, drinks_per_day)

    x = pd.to_numeric(drinks_per_day, errors="coerce")
    alc_new = pd.Series(pd.NA, index=work.index, dtype="Int64")
    alc_new = alc_new.mask((x >= 0) & (x < rare_cutoff_per_day), 1)  # none/rare
    alc_new = alc_new.mask((work["RIAGENDR"] == 1) & (x >= rare_cutoff_per_day) & (x < 2), 2)
    alc_new = alc_new.mask((work["RIAGENDR"] == 2) & (x >= rare_cutoff_per_day) & (x < 1), 2)
    alc_new = alc_new.mask((work["RIAGENDR"] == 1) & (x >= 2), 3)
    alc_new = alc_new.mask((work["RIAGENDR"] == 2) & (x >= 1), 3)

    if replace_existing:
        out.loc[idx, "alcg2"] = alc_new.values
    else:
        need = out.loc[idx, "alcg2"].isna()
        out.loc[idx[need], "alcg2"] = alc_new.loc[idx[need]].values

    return out

# ---------------------------
# Weighted % by cycle (auto weight: 66→WTMECPRP else→WTMEC2YR)
# ---------------------------
def _weighted_alcg2_for_cycle(df, cycle, weight_col):
    a = df.copy()
    a["SDDSRVYR"] = pd.to_numeric(a["SDDSRVYR"], errors="coerce")
    a = a[(a["SDDSRVYR"] == cycle) & (pd.to_numeric(a["RIDAGEYR"], errors="coerce") >= 18)]
    if a.empty:
        return pd.Series({1: np.nan, 2: np.nan, 3: np.nan, "NA": np.nan}, name=cycle)
    w = pd.to_numeric(a.get(weight_col), errors="coerce")
    denom = np.nansum(w)
    if not np.isfinite(denom) or denom <= 0:
        return pd.Series({1: np.nan, 2: np.nan, 3: np.nan, "NA": np.nan}, name=cycle)
    return pd.Series({
        1: float(np.nansum(w[a["alcg2"] == 1]) / denom),
        2: float(np.nansum(w[a["alcg2"] == 2]) / denom),
        3: float(np.nansum(w[a["alcg2"] == 3]) / denom),
        "NA": float(np.nansum(w[a["alcg2"].isna()]) / denom),
    }, name=cycle)

def alcg2_by_cycle_auto_weight(df, include_cycles=None):
    x = df.copy()
    x["SDDSRVYR"] = pd.to_numeric(x["SDDSRVYR"], errors="coerce")
    cycles = sorted(x["SDDSRVYR"].dropna().unique().tolist())
    if include_cycles is not None:
        cycles = [c for c in cycles if c in include_cycles]
    rows = []
    for cyc in cycles:
        weight = "WTMECPRP" if cyc == 66 else "WTMEC2YR"
        rows.append(_weighted_alcg2_for_cycle(x, cyc, weight))
    res = pd.DataFrame(rows).sort_index()
    print("=== alcg2 % by cycle (weighted; auto weight per cycle) ===")
    print((res * 100).round(1).to_string())
    return res

# ---------------------------
# USAGE
# ---------------------------
# 1) start from your DF: df_my_cov_aligned_short
# 2) (optional) ensure gender/age for 66 are present
df_my_cov_aligned_short = attach_p_demo_gender(df_my_cov_aligned_short, replace_existing=False)

# 3) compute alcg2 for 2017–Mar 2020 WITHOUT adding ALQ columns
df_my_cov_aligned_short = update_alcg2_for_p_cycle_no_intermediates(
    df_my_cov_aligned_short,
    replace_existing=False,
    rare_cutoff_per_day=0.03,
    backfill_alq130_if_freq=True
)

# 4) quick sanity check that intermediates were NOT added
print([c for c in ["ALQ111","ALQ121","ALQ130"] if c in df_my_cov_aligned_short.columns])  # expect []

# 5) optional weighted table (66 uses WTMECPRP automatically)
_ = alcg2_by_cycle_auto_weight(df_my_cov_aligned_short)


[]
=== alcg2 % by cycle (weighted; auto weight per cycle) ===
         1     2     3    NA
1.0   34.0  39.5   9.8  16.7
2.0   25.9  39.8   8.7  25.5
3.0   32.8  38.7   7.6  20.9
4.0   31.6  39.6   9.7  19.0
5.0   31.2  39.5   8.1  21.2
6.0   26.3  41.4   9.6  22.7
7.0   26.3  44.7  10.1  18.9
8.0   31.0  45.0   8.6  15.4
9.0   30.1  45.7   8.5  15.7
10.0  25.9  44.4   8.6  21.1
12.0  23.2  42.3   9.6  24.8
66.0  24.9  45.7   9.3  20.2


In [73]:
df_my_cov_aligned_short.columns

Index(['SEQN', 'SDDSRVYR', 'sdmvpsu', 'sdmvstra', 'RIDAGEYR', 'SEX', 'RACE',
       're', 'household_size', 'DMDHHSIZ', 'EDU', 'pir', 'SMK_AVG', 'SMK',
       'met_hr', 'SMK_STATUS', 'CIGS_PER_DAY', 'PACK_YEARS', 'FORMER_SMOKER',
       'METSCORE', 'LTPA', 'bmi_cat', 'BMI_CLAS', 'DIABE', 'HYPERTEN',
       'chol_rx', 'CVD', 'cancer', 'probable_depression', 'ahei_total',
       'unemployment2', 'ins', 'HOQ065', 'marriage', 'SNAP', 'FS', 'WTINT2YR',
       'WTMEC2YR', 'WTSAF2YR', 'WTINT4YR', 'WTMEC4YR', 'WTPH2YR', 'WTINTPRP',
       'WTMECPRP', 'WTSAFPRP', 'wt_int', 'wt_mec', 'wt_fasting',
       'wt_phlebotomy', 'marriage_prev', 'marriage_label', 'marriage3',
       'SNAP_src', 'SNAP_bin', 'SNAP_src_rank', 'SNAP_indiv_only',
       'SNAP_indiv_plus_singleton', 'bmi', 'HOQ065_structural_missing',
       'household_size_structural_missing', 'chol_rx_structural_missing',
       'RIAGENDR', 'drinking', 'alcg2', 'perE_alco', 'METSCORE_fromPAQ',
       'LTPA_fromPAQ', 'met_hr_recalc_from', 'c

In [74]:
# df_my_cov_aligned_short = df_my_cov_aligned_short.drop(columns=["ALQ111","ALQ121","ALQ130"], errors="ignore")


## save as cov_addv5_99_23

In [76]:
from pathlib import Path

out_dir = Path("/Users/dengshuyue/Desktop/SDOH/analysis/output")
out_dir.mkdir(parents=True, exist_ok=True)

# Parquet (preferred; needs `pyarrow` or `fastparquet`)
df_my_cov_aligned_short.to_parquet(out_dir / "cov_addv5_99_23.parquet", index=False)

# CSV fallback
# df_my_cov_aligned_short.to_csv(out_dir / "cov_addv5_99_23.csv", index=False)
